# Part II - VacationPyVacationPy
----
Use your skills in working with weather data to plan future vacations. Use jupyter-gmaps and the Google Places API for this part of the assignment.

* Create a heat map that displays the humidity for every city from Part I.

* Narrow down the DataFrame to find your ideal weather condition. For example:

  * A max temperature lower than 80 degrees but higher than 70.

  * Wind speed less than 10 mph.

  * Zero cloudiness.

  * Drop any rows that don't contain all three conditions. You want to be sure the weather is ideal.

  * **Note:** Feel free to adjust to your specifications but be sure to limit the number of rows returned by your API requests to a reasonable number.

* Using Google Places API to find the first hotel for each city located within 5000 meters of your coordinates.

* Plot the hotels on top of the humidity heatmap with each pin containing the **Hotel Name**, **City**, and **Country**.

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.
* If you having trouble displaying the maps, try running `jupyter nbextension enable --py gmaps` in your environment and retry.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests as rq
import gmaps
import os
from pprint import pprint
# Import API key
from config import google_api

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [13]:
# Load the csv exported in Part I to a DataFrame
cities_df = pd.read_csv('../output_data/cities2.csv')
cities_df

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max_Temp,Wind_Speed
0,2122614,Okha,100,RU,1612330021,84,53.5739,142.9478,-21.14,20.20
1,5106841,Albany,98,US,1612329967,87,42.6001,-73.9662,27.00,9.33
2,3466704,Castro,96,BR,1612330021,94,-24.7911,-50.0119,64.54,3.20
3,1015776,Bredasdorp,0,ZA,1612330021,94,-34.5322,20.0403,60.80,2.30
4,2961459,Skibbereen,0,IE,1612330021,93,51.5500,-9.2667,37.40,9.22
...,...,...,...,...,...,...,...,...,...,...
568,2446796,Bilma,0,NE,1612330090,35,18.6853,12.9164,57.67,10.92
569,5101798,Newark,90,US,1612330090,74,40.7357,-74.1724,32.00,8.05
570,1529376,Korla,1,CN,1612330090,64,41.7597,86.1469,35.26,2.91
571,1121143,Zabul Province,0,AF,1612330090,68,32.2500,67.2500,39.76,3.89


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=google_api)

In [20]:
#use lat&lng as locations for heat layer
locations = cities_df[['Lat','Lng']]
#use humidity to create the weights 
humid = cities_df['Humidity']
humidmax = cities_df['Humidity'].max()
gradient = [
    "rgba(0, 255, 255, 0)",
    "rgba(0, 255, 255, 1)",
    "rgba(0, 191, 255, 1)",
    "rgba(0, 127, 255, 1)",
    "rgba(0, 63, 255, 1)",
    "rgba(0, 0, 255, 1)",
    "rgba(0, 0, 223, 1)",
    "rgba(0, 0, 191, 1)",
    "rgba(0, 0, 159, 1)",
    "rgba(0, 0, 127, 1)",
    "rgba(63, 0, 91, 1)",
    "rgba(127, 0, 63, 1)",
    "rgba(191, 0, 31, 1)",
    "rgba(255, 0, 0, 1)",
  ]

In [21]:
#setup your figure dimensions
figure_layout = {'width': '400px','height': '300px',
                 'border': '1px solid black',
                 'padding': '1px','margin': '0 auto 0 auto'}
#create figure using the layout from above
fig = gmaps.figure(layout=figure_layout)

# create heat layer and formatting
heat = gmaps.heatmap_layer(locations, weights=humid, dissipating=False, max_intensity=humidmax, point_radius=3,
    gradient=gradient,)

#add heat layer to map figure
fig.add_layer(heat)

fig

Figure(layout=FigureLayout(border='1px solid black', height='300px', margin='0 auto 0 auto', padding='1px', wi…

### Create new DataFrame fitting weather criteria
* Narrow down the DataFrame to find your ideal weather condition.

  * A max temperature lower than 75 degrees but higher than 60.

  * Wind speed less than 10 mph.

  * Humidity is less than 50%.
  
* Drop any rows that don't contain all three conditions, or are null

In [6]:
# make a copy of df
ideal_hotels = np.where((cities_df.Max_Temp < 70) & (cities_df.Max_Temp > 60) & (cities_df.Wind_Speed < 10) & (cities_df.Humidity < 50))
ideal_df = cities_df.loc[ideal_hotels]
ideal_df

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max_Temp,Wind_Speed
104,2451184,Sikasso,0,ML,1612330033,23,10.9167,-7.0000,65.88,4.90
139,293842,Kiryat Gat,19,IL,1612330037,22,31.6100,34.7642,61.00,6.17
212,2155415,New Norfolk,14,AU,1612330046,35,-42.7826,147.0587,68.00,4.00
233,2427637,Mongo,0,TD,1612330049,22,12.1844,18.6930,65.79,5.75
237,2221513,Tignère,0,CM,1612330049,30,7.3667,12.6500,62.98,1.70
264,2381334,Atar,16,MR,1612330052,24,20.5169,-13.0499,65.34,6.31
266,2427455,Moundou,0,TD,1612330052,23,8.5667,16.0833,68.83,3.11
282,244878,Biltine,0,TD,1612330054,21,14.5333,20.9167,67.12,8.57
321,287832,Ibrā’,43,OM,1612330059,42,22.6906,58.5334,69.84,0.89
323,1183880,Bahawalpur,2,PK,1612330060,24,29.4000,71.6833,68.02,5.99


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Using Google Places API to find the first hotel for each city located within 5000 meters of your coordinates.
    * Set parameters to search for hotels with 5000 meters.
    * Hit the Google Places API for each city's coordinates.
    * Store the first Hotel result into the DataFrame.
    * Plot the hotels on top of the humidity heatmap with each pin containing the **Hotel Name**, **City**, and **Country**

In [7]:
# make a copy and add a new column for hotel name
hotel_df = ideal_df.copy()
hotel_df['Hotel_Name'] = ''
#create the base url
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
#define the variables to use in our api call
params = {'type': 'lodging', 'radius':5000,'key':google_api,}
hotel_df

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max_Temp,Wind_Speed,Hotel_Name
104,2451184,Sikasso,0,ML,1612330033,23,10.9167,-7.0000,65.88,4.90,
139,293842,Kiryat Gat,19,IL,1612330037,22,31.6100,34.7642,61.00,6.17,
212,2155415,New Norfolk,14,AU,1612330046,35,-42.7826,147.0587,68.00,4.00,
233,2427637,Mongo,0,TD,1612330049,22,12.1844,18.6930,65.79,5.75,
237,2221513,Tignère,0,CM,1612330049,30,7.3667,12.6500,62.98,1.70,
264,2381334,Atar,16,MR,1612330052,24,20.5169,-13.0499,65.34,6.31,
266,2427455,Moundou,0,TD,1612330052,23,8.5667,16.0833,68.83,3.11,
282,244878,Biltine,0,TD,1612330054,21,14.5333,20.9167,67.12,8.57,
321,287832,Ibrā’,43,OM,1612330059,42,22.6906,58.5334,69.84,0.89,
323,1183880,Bahawalpur,2,PK,1612330060,24,29.4000,71.6833,68.02,5.99,


In [8]:
for i, row in hotel_df.iterrows():
    lat = row['Lat']
    lng = row['Lng']
    params['location'] = f'{lat}, {lng}'
    response = rq.get(base_url,params=params).json()
    try:
        hotel_df.loc[i,'Hotel_Name'] = response['results'][0]['name']
    except:
        hotel_df.loc[i,'Hotel_Name'] = 'Hotel not in radius'        

In [9]:
hotel_df

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max_Temp,Wind_Speed,Hotel_Name
104,2451184,Sikasso,0,ML,1612330033,23,10.9167,-7.0000,65.88,4.90,Hotel not in radius
139,293842,Kiryat Gat,19,IL,1612330037,22,31.6100,34.7642,61.00,6.17,OlusHome
212,2155415,New Norfolk,14,AU,1612330046,35,-42.7826,147.0587,68.00,4.00,The Shingles Riverside Cottages
233,2427637,Mongo,0,TD,1612330049,22,12.1844,18.6930,65.79,5.75,Résidences du Programme FIDA
237,2221513,Tignère,0,CM,1612330049,30,7.3667,12.6500,62.98,1.70,SADECK
264,2381334,Atar,16,MR,1612330052,24,20.5169,-13.0499,65.34,6.31,Oumou Elghoura
266,2427455,Moundou,0,TD,1612330052,23,8.5667,16.0833,68.83,3.11,Planete Life
282,244878,Biltine,0,TD,1612330054,21,14.5333,20.9167,67.12,8.57,Hôtel Trois Étoiles de Biltine
321,287832,Ibrā’,43,OM,1612330059,42,22.6906,58.5334,69.84,0.89,مزرعة راحة النفوس المنزفة
323,1183880,Bahawalpur,2,PK,1612330060,24,29.4000,71.6833,68.02,5.99,5th Avenue Hotel


In [23]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel_Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [24]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(border='1px solid black', height='300px', margin='0 auto 0 auto', padding='1px', wi…